https://health.google.com/covid-19/open-data/raw-data

https://github.com/GoogleCloudPlatform/covid-19-open-data#use-the-data


In [1]:
import pandas as pd
import plotly.express as px

18 datasets. Aggregated has all of them merged together.

# Epidemiology (outcome)

In [2]:
df_epidemiology = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv")
df_epidemiology.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
0,2020-01-01,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
1,2020-01-02,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
2,2020-01-03,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
3,2020-01-04,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
4,2020-01-05,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [3]:
df_epidemiology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12525825 entries, 0 to 12525824
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   date                  object 
 1   location_key          object 
 2   new_confirmed         float64
 3   new_deceased          float64
 4   new_recovered         float64
 5   new_tested            float64
 6   cumulative_confirmed  float64
 7   cumulative_deceased   float64
 8   cumulative_recovered  float64
 9   cumulative_tested     float64
dtypes: float64(8), object(2)
memory usage: 955.6+ MB


In [4]:
df_epidemiology.tail()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
12525820,2022-09-09,ZW,11.0,0.0,NaN,NaN,256870.0,5596.0,NaN,NaN
12525821,2022-09-10,ZW,18.0,0.0,NaN,NaN,256888.0,5596.0,NaN,NaN
12525822,2022-09-11,ZW,0.0,0.0,NaN,NaN,256888.0,5596.0,NaN,NaN
12525823,2022-09-12,ZW,51.0,0.0,NaN,NaN,256939.0,5596.0,NaN,NaN
12525824,2022-09-13,ZW,0.0,0.0,NaN,NaN,256939.0,5596.0,NaN,NaN


In [5]:
df_epidemiology.isna().sum()

date                          0
location_key                987
new_confirmed             50025
new_deceased             858687
new_recovered           8545363
new_tested              9331336
cumulative_confirmed     198780
cumulative_deceased     1051000
cumulative_recovered    8534668
cumulative_tested       9512905
dtype: int64

1. Drop rows with `Nan` for `location_key`
2. Change `date` to datetime
3. Cumulative vs. New per day (target variable)
4. Country
5. Day or Month
6. TimeSeries for machine learning

In [6]:
# drop rows with null values for location_key
df_epidemiology = df_epidemiology[df_epidemiology['location_key'].notna()]

In [8]:
# Read index file
df_index = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/index.csv")
df_index.head()

,location_key,place_id,wikidata_id,datacommons_id,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level
0,AD,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,AD,AND,0
1,AE,ChIJvRKrsd9IXj4RpwoIwFYv0zM,Q878,country/ARE,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,AE,ARE,0
2,AF,ChIJbQL_-LZu0TgReNqWvg1GtfM,Q889,country/AFG,AF,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,AF,AFG,0
3,AF_BAL,ChIJWZOescNSyzgRkmVJ8fmfR3k,Q121104,NaN,AF,Afghanistan,BAL,Balkh,NaN,NaN,NaN,NaN,AF,AFG,1
4,AF_BAM,ChIJw-bKQwDr0jgRQiw8jHsQ-HI,Q171382,NaN,AF,Afghanistan,BAM,Bamyan,NaN,NaN,NaN,NaN,AF,AFG,1


In [9]:
df_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22963 entries, 0 to 22962
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   location_key        22962 non-null  object
 1   place_id            21661 non-null  object
 2   wikidata_id         22184 non-null  object
 3   datacommons_id      4216 non-null   object
 4   country_code        22962 non-null  object
 5   country_name        22963 non-null  object
 6   subregion1_code     22715 non-null  object
 7   subregion1_name     22716 non-null  object
 8   subregion2_code     21253 non-null  object
 9   subregion2_name     21253 non-null  object
 10  locality_code       32 non-null     object
 11  locality_name       32 non-null     object
 12  iso_3166_1_alpha_2  22961 non-null  object
 13  iso_3166_1_alpha_3  22962 non-null  object
 14  aggregation_level   22963 non-null  int64 
dtypes: int64(1), object(14)
memory usage: 2.6+ MB


In [10]:
df_index.isna().sum()

location_key              1
place_id               1302
wikidata_id             779
datacommons_id        18747
country_code              1
country_name              0
subregion1_code         248
subregion1_name         247
subregion2_code        1710
subregion2_name        1710
locality_code         22931
locality_name         22931
iso_3166_1_alpha_2        2
iso_3166_1_alpha_3        1
aggregation_level         0
dtype: int64

In [43]:
df_index = df_index[df_index['location_key'].notna()]

In [44]:
# Just focusing on rows with cumulative country codes without subregions
df_index_cumul = df_index[df_index['subregion1_code'].isna()]

In [45]:
cumul_country_code = list(df_index_cumul['location_key'].unique())
cumul_country_code

['AD',
 'AE',
 'AF',
 'AG',
 'AI',
 'AL',
 'AM',
 'AN',
 'AO',
 'AQ',
 'AR',
 'AS',
 'AT',
 'AU',
 'AW',
 'AZ',
 'BA',
 'BB',
 'BD',
 'BE',
 'BF',
 'BG',
 'BH',
 'BI',
 'BJ',
 'BM',
 'BN',
 'BO',
 'BQ',
 'BR',
 'BS',
 'BT',
 'BV',
 'BW',
 'BY',
 'BZ',
 'CA',
 'CC',
 'CD',
 'CF',
 'CG',
 'CH',
 'CI',
 'CK',
 'CL',
 'CM',
 'CN',
 'CO',
 'CR',
 'CU',
 'CV',
 'CW',
 'CX',
 'CY',
 'CZ',
 'DE',
 'DJ',
 'DK',
 'DM',
 'DO',
 'DZ',
 'EC',
 'EE',
 'EG',
 'EH',
 'ER',
 'ES',
 'ET',
 'FI',
 'FJ',
 'FK',
 'FM',
 'FO',
 'FR',
 'GA',
 'GB',
 'GD',
 'GE',
 'GF',
 'GG',
 'GH',
 'GI',
 'GL',
 'GM',
 'GN',
 'GQ',
 'GR',
 'GS',
 'GT',
 'GU',
 'GW',
 'GY',
 'HK',
 'HM',
 'HN',
 'HR',
 'HT',
 'HU',
 'ID',
 'IE',
 'IL',
 'IM',
 'IN',
 'IO',
 'IQ',
 'IQ_NA',
 'IR',
 'IS',
 'IT',
 'JE',
 'JM',
 'JO',
 'JP',
 'KE',
 'KG',
 'KH',
 'KI',
 'KM',
 'KN',
 'KP',
 'KR',
 'KW',
 'KY',
 'KZ',
 'LA',
 'LB',
 'LC',
 'LI',
 'LK',
 'LR',
 'LS',
 'LT',
 'LU',
 'LV',
 'LY',
 'MA',
 'MC',
 'MD',
 'ME',
 'MG',
 'MH',
 'MK',
 'M

In [19]:
# Getting the cumulative US row
df_index_cumul.loc[df_index['country_name'] == 'United States of America']

,location_key,place_id,wikidata_id,datacommons_id,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level
19557,US,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,Q30,country/USA,US,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,US,USA,0


# Exploration (US only)

In [27]:
df_us_epidemiology = df_epidemiology[df_epidemiology['location_key'] == 'US'].reset_index(drop=True)
df_us_epidemiology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  988 non-null    object 
 1   location_key          988 non-null    object 
 2   new_confirmed         988 non-null    float64
 3   new_deceased          988 non-null    float64
 4   new_recovered         0 non-null      float64
 5   new_tested            840 non-null    float64
 6   cumulative_confirmed  988 non-null    float64
 7   cumulative_deceased   988 non-null    float64
 8   cumulative_recovered  0 non-null      float64
 9   cumulative_tested     840 non-null    float64
dtypes: float64(8), object(2)
memory usage: 77.3+ KB


In [28]:
df_us_epidemiology.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
0,2020-01-01,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
1,2020-01-02,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
2,2020-01-03,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
3,2020-01-04,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
4,2020-01-05,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [29]:
# date column to datetime type
df_us_epidemiology['date'] = pd.to_datetime(df_us_epidemiology['date'], format='%Y-%m-%d')
df_us_epidemiology['date'].dtype

dtype('<M8[ns]')

In [30]:
df_us_epidemiology.describe()

,date,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
count,988,9.880000e+02,988.00000,0.0,8.400000e+02,9.880000e+02,9.880000e+02,0.0,8.400000e+02
mean,2021-05-08 12:00:00.000000256,9.290546e+04,1000.02834,NaN,1.086630e+06,3.557368e+07,5.036650e+05,NaN,4.103864e+08
min,2020-01-01 00:00:00,0.000000e+00,0.00000,NaN,3.480000e+02,0.000000e+00,0.000000e+00,NaN,3.480000e+02
25%,2020-09-03 18:00:00,2.699375e+04,379.25000,NaN,6.613455e+05,6.040394e+06,1.705235e+05,NaN,1.160973e+08
50%,2021-05-08 12:00:00,5.583850e+04,752.00000,NaN,1.011313e+06,3.160353e+07,5.505225e+05,NaN,4.163406e+08
75%,2022-01-10 06:00:00,1.194122e+05,1419.50000,NaN,1.496003e+06,6.010643e+07,8.054482e+05,NaN,6.603548e+08
max,2022-09-14 00:00:00,1.235521e+06,4133.00000,NaN,3.201706e+06,9.244050e+07,1.005195e+06,NaN,9.127691e+08
std,NaN,1.264354e+05,865.17391,NaN,5.971515e+05,3.039155e+07,3.415858e+05,NaN,2.972203e+08


In [31]:
outcome_col = ["new_confirmed", "new_deceased", "cumulative_confirmed", "cumulative_deceased"]

df_us_epidemiology[outcome_col].isna().sum()

new_confirmed           0
new_deceased            0
cumulative_confirmed    0
cumulative_deceased     0
dtype: int64

In [32]:
# US epidemiology
fig = px.line(df_us_epidemiology, x='date', y=outcome_col,
              width=1000, height=600, title='Number of COVID-19 Patients in US Across Time')
fig.show()

In [33]:
# Same thing in histogram, and only for the newly confirmed patients
fig = px.histogram(df_us_epidemiology, x="date", y='new_confirmed', nbins=60, 
                   histfunc="avg", width=1000, height=600, 
                   title="Histogram on Average Newly Confirmed Patients in the US")
fig.show()

In [68]:
# Export clean CSV file
df_us_epidemiology[['date', "new_confirmed", "new_deceased", "cumulative_confirmed", "cumulative_deceased"]].to_csv("US_epidemiology.csv")

# Demographic

In [35]:
demographics_df = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/demographics.csv")
demographics_df.head()

,location_key,population,population_male,population_female,population_rural,population_urban,population_largest_city,population_clustered,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older
0,AD,77265.0,58625.0,55581.0,9269.0,67873.0,NaN,NaN,164.394,0.858,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0
1,AE,9890400.0,6836349.0,3054051.0,1290785.0,8479744.0,2833079.0,5914068.0,118.306,0.863,1011713.0,842991.0,2149343.0,3169314.0,1608109.0,797913.0,242707.0,55884.0,12426.0
2,AF,38928341.0,19976265.0,18952076.0,28244481.0,9797273.0,4114030.0,4114030.0,59.627,0.498,11088732.0,9821559.0,7035871.0,4534646.0,2963459.0,1840198.0,1057496.0,480455.0,105925.0
3,AF_BAL,1509183.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AF_BAM,495557.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
demographics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21689 entries, 0 to 21688
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   location_key                 21688 non-null  object 
 1   population                   21659 non-null  float64
 2   population_male              17607 non-null  float64
 3   population_female            17607 non-null  float64
 4   population_rural             213 non-null    float64
 5   population_urban             213 non-null    float64
 6   population_largest_city      152 non-null    float64
 7   population_clustered         121 non-null    float64
 8   population_density           1192 non-null   float64
 9   human_development_index      5738 non-null   float64
 10  population_age_00_09         17249 non-null  float64
 11  population_age_10_19         17248 non-null  float64
 12  population_age_20_29         17248 non-null  float64
 13  population_age_3

In [46]:
demographics_cumul_df = demographics_df[demographics_df['location_key'].isin(cumul_country_code)]
demographics_cumul_df.info()
# need to re-visit this because there are a few location_keys that are not cumul.

<class 'pandas.core.frame.DataFrame'>
Index: 247 entries, 0 to 21688
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   location_key                 247 non-null    object 
 1   population                   246 non-null    float64
 2   population_male              235 non-null    float64
 3   population_female            235 non-null    float64
 4   population_rural             212 non-null    float64
 5   population_urban             212 non-null    float64
 6   population_largest_city      151 non-null    float64
 7   population_clustered         121 non-null    float64
 8   population_density           230 non-null    float64
 9   human_development_index      185 non-null    float64
 10  population_age_00_09         235 non-null    float64
 11  population_age_10_19         235 non-null    float64
 12  population_age_20_29         235 non-null    float64
 13  population_age_30_39   

1. consider `population_density` and other population on different age range as features to keep.
2. Timeseries and forecasting as part 1 for some countries of interest
3. Predictive modeling for these countries on cumulative cases and deaths with various features